# Algorytmy macierzowe - zadanie nr 3 - Formaty macierzy rzadkich

" Proszę wybrać swój ulubiony język programowania. Proszę wziąć największe macierze, które używali Państwo w zadaniu pierwszym (ewentualnie zmniejszyć p żeby stały się rzadsze). Proszę przekonwertować je do stosownego formatu i przemnożyć
przez siebie.

6. Pierwsza i druga macierz CSR format "

Marcin Hawryluk, Norbert Wolniak <br>grupa: piątek 12:50B <hr>

In [2]:
import numpy as np
from time import time
import pandas as pd
import os

## Generowanie macierzy

In [2]:
def read_matrix(file_name):
    with open(file_name, 'r') as file:
        for line in file:
            if line.strip() == '':
                continue
            if line[0] == '#':
                if line[2:6] == "rows":
                    _, _, size = line.split()
                    size = int(size)
                    matrix = np.zeros((size, size))
            else:
                row, col, val = line.split(' ')
                matrix[int(row)-1, int(col)-1] = val
            
    return matrix

## Format CSR

## Mnożenie macierzy

## Pomiar czasów

## Wnioski

* 
* 

M. Hawryluk, N. Wolniak. 2021